In [36]:
# Import required packages
import numpy as np
import pandas as pd
from sklearn.decomposition import FastICA
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import os
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.decomposition import PCA

In [25]:
from keras.models import Sequential
from keras.layers import Dense,Flatten,LSTM, GRU
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.metrics import RootMeanSquaredError,mean_absolute_percentage_error
from keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dropout
from keras.models import Model
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [26]:
# split a univariate time series into patterns
def get_Patterns(TSeries, n_inputs,h):
    X,y,z = pd.DataFrame(np.zeros((len(TSeries)-n_inputs-h+1,n_inputs))), pd.DataFrame(), pd.DataFrame()
    for i in range(len(TSeries)):
        # find the end of this pattern
        end_ix = i + n_inputs + h - 1
        # check if we are beyond the time series
        if end_ix > len(TSeries)-1:
            break
        # gather input and output parts of the pattern
        for j in range(n_inputs):
            X.loc[i,j]=TSeries.iloc[i+j,0]
        i=i+n_inputs
        y=y.append(TSeries.iloc[end_ix], ignore_index = True)
    return np.array(X),np.array(y)

In [27]:
# originalData should be a Column Vectored DataFrame
def minmaxNorm(originalData, lenTrainValidation):
    max2norm=max(originalData.iloc[0:lenTrainValidation,0])
    min2norm=min(originalData.iloc[0:lenTrainValidation,0])
    lenOriginal=len(originalData)
    normalizedData=np.zeros(lenOriginal)   
    normalizedData = []
    for i in range (lenOriginal):
        normalizedData.append((originalData.iloc[i]-min2norm)/(max2norm-min2norm))    
    return pd.DataFrame(normalizedData)

In [28]:
# originalData and forecastedData should be Column Vectored DataFrames
def minmaxDeNorm( originalData, forecastedData, lenTrainValidation):
    max2norm=max(originalData.iloc[0:lenTrainValidation,0])
    min2norm=min(originalData.iloc[0:lenTrainValidation,0])
    lenOriginal=len(originalData)
    denormalizedData=[]    
    for i in range (lenOriginal):
        denormalizedData.append((forecastedData.iloc[i]*(max2norm-min2norm))+min2norm)  
    return pd.DataFrame(denormalizedData)

In [29]:
# Timeseries_Data and forecasted_value should be Column Vectored DataFrames
def findRMSE( Timeseries_Data, forecasted_value,lenTrainValidation):
    l=Timeseries_Data.shape[0]
    lenTest=l-lenTrainValidation
    trainRMSE=0;
    for i in range (lenTrainValidation):
        trainRMSE=trainRMSE+np.power((forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0]),2) 
    trainRMSE=np.sqrt(trainRMSE/lenTrainValidation)

    testRMSE=0;
    for i in range (lenTrainValidation,l,1):
        testRMSE=testRMSE+np.power((forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0]),2)
    testRMSE=np.sqrt(testRMSE/lenTest)
    return trainRMSE, testRMSE 

In [30]:
# Timeseries_Data and forecasted_value should be Column Vectored DataFrames
def findMAE( Timeseries_Data, forecasted_value,lenTrainValidation):
    l=Timeseries_Data.shape[0]
    lenTest=l-lenTrainValidation
    trainMAE=0;
    for i in range (lenTrainValidation):
        trainMAE=trainMAE+np.abs(forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0]) 
    trainMAE=(trainMAE/(lenTrainValidation));

    testMAE=0;
    for i in range (lenTrainValidation,l,1):
        testMAE=testMAE+np.abs(forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0])
    testMAE=(testMAE/lenTest);
    return trainMAE, testMAE

In [31]:
def Find_Features_LSTM(x,y,lenValid,lenTest):
    NOP=len(y)
    n_features=1
    lenTrain=NOP-lenValid-lenTest
    LagLength=x.shape[1]
    xTrain=x[0:lenTrain,:]
    xValid=x[lenTrain:(lenTrain+lenValid),:]
    xTest=x[(lenTrain+lenValid):NOP,:]
    yTrain=y[0:lenTrain,0]
    yValid=y[lenTrain:(lenTrain+lenValid),0]
    yTest=y[(lenTrain+lenValid):NOP,0]
    model1 = Sequential()
    model1.add(LSTM(256, activation='tanh', input_shape=(LagLength,n_features),kernel_initializer="he_normal",name="layer1"))
    model1.add(Flatten())
    model1.add(Dense(16, activation='relu',kernel_initializer="he_normal"))
    model1.add(Dense(1,activation='linear'))
    checkpoint_filepath = './'
    cp1=ModelCheckpoint(checkpoint_filepath,save_best_only='True')
    model1.compile(loss=MeanSquaredError(),optimizer=Adam(learning_rate=0.01),metrics=[RootMeanSquaredError()])
    # fit
    model1.fit(xTrain, yTrain,validation_data=(xValid,yValid), epochs=100, verbose=1,batch_size=16)
    yhatNorm=model1.predict(x).flatten().reshape(x.shape[0],1)
    feature_extractor = Model(inputs=model1.inputs, outputs=model1.get_layer(name="layer1").output)
    features = pd.DataFrame(feature_extractor(x))
    return features

In [32]:
def Find_Features_GRU(x,y,lenValid,lenTest):
    NOP=len(y)
    n_features=1
    lenTrain=NOP-lenValid-lenTest
    LagLength=x.shape[1]
    xTrain=x[0:lenTrain,:]
    xValid=x[lenTrain:(lenTrain+lenValid),:]
    xTest=x[(lenTrain+lenValid):NOP,:]
    yTrain=y[0:lenTrain,0]
    yValid=y[lenTrain:(lenTrain+lenValid),0]
    yTest=y[(lenTrain+lenValid):NOP,0]
    model2 = Sequential()
    model2.add(GRU(256, activation='tanh', input_shape=(LagLength,n_features),kernel_initializer="he_normal",name="layer1"))
    model2.add(Flatten())
    model2.add(Dense(16, activation='relu',kernel_initializer="he_normal"))
    model2.add(Dense(1,activation='linear'))
    checkpoint_filepath = './'
    cp2=ModelCheckpoint(checkpoint_filepath,save_best_only='True')
    model2.compile(loss=MeanSquaredError(),optimizer=Adam(learning_rate=0.01),metrics=[RootMeanSquaredError()])
    # fit
    model2.fit(xTrain, yTrain,validation_data=(xValid,yValid), epochs=100, verbose=1,batch_size=16)
    yhatNorm=model2.predict(x).flatten().reshape(x.shape[0],1)
    feature_extractor = Model(inputs=model2.inputs, outputs=model2.get_layer(name="layer1").output)
    features = pd.DataFrame(feature_extractor(x))
    return features

In [33]:
def Find_Fitness(x,y,lenValid,lenTest,model):
    NOP=y.shape[0]
    lenTrain=NOP-lenValid-lenTest
    xTrain=x[0:(lenTrain+lenValid),:]
    xValid=x[lenTrain:(lenTrain+lenValid),:]
    xTest=x[(lenTrain+lenValid):NOP,:]
    yTrain=y[0:(lenTrain+lenValid),0]
    yValid=y[lenTrain:(lenTrain+lenValid),0]
    yTest=y[(lenTrain+lenValid):NOP,0]
    model.fit(xTrain, yTrain)
    yhatNorm=model.predict(x).flatten().reshape(x.shape[0],1)
    return pd.DataFrame(yhatNorm)

In [37]:
#Read the Time Series Dataset
Timeseries_Data=pd.read_csv('AQI.csv',header=None)
LagLength=24
h=1
NumberOfRepeatitions=2
lt=Timeseries_Data.shape[0]
lenTrain=int(round(lt*0.6))
lenValidation=int(round(lt*0.2))
lenTest=int(lt-lenTrain-lenValidation)
# NORMALIZE THE DATA
normalizedData=minmaxNorm(Timeseries_Data,lenTrain+lenValidation);
# Transform the Time Series into Patterns Using Sliding Window
X, y = get_Patterns(normalizedData, LagLength, h)
LSTMFeatures=Find_Features_LSTM(X,y,lenValidation,lenTest)
GRUFeatures=Find_Features_GRU(X,y,lenValidation,lenTest)
X=np.array(np.hstack((LSTMFeatures,GRUFeatures)))
#X = SelectKBest(score_func=f_regression, k=24).fit_transform(X, y)
ica = FastICA(n_components=24)
X = ica.fit_transform(X)
MeanAccuracyT=pd.DataFrame()
StdAccuracyT=pd.DataFrame()
model=RandomForestRegressor()
name='9.RandomForestRegressor'
file1='./'+str(name)+"_Accuracy.xlsx"
file2='./'+str(name)+"_Forecasts.xlsx"
Forecasts=pd.DataFrame()
Accuracy=pd.DataFrame()
ynorm1=Find_Fitness(X,y,lenValidation,lenTest,model)
ynorm=pd.DataFrame(normalizedData.iloc[0:LagLength,0])
ynorm=ynorm.append(ynorm1,ignore_index = True)
yhat=minmaxDeNorm(Timeseries_Data, ynorm, lenTrain+lenValidation)
Accuracy.loc[0,0],Accuracy.loc[0,1]=findRMSE( Timeseries_Data,yhat,lenTrain+lenValidation)
Accuracy.loc[0,2],Accuracy.loc[0,3]=findMAE( Timeseries_Data,yhat,lenTrain+lenValidation)
Forecasts=Forecasts.append(yhat.T,ignore_index = True)
Accuracy.to_excel(file1,sheet_name='Accuracy',index=False)
Forecasts.T.to_excel(file2,sheet_name='Forecasts',index=False)
Accuracy

Epoch 1/100
18/18 [==============================] - 7s 156ms/step - loss: 1.4600 - root_mean_squared_error: 1.2083 - val_loss: 0.0061 - val_root_mean_squared_error: 0.0782
Epoch 2/100
18/18 [==============================] - 2s 98ms/step - loss: 0.0273 - root_mean_squared_error: 0.1653 - val_loss: 0.0030 - val_root_mean_squared_error: 0.0545
Epoch 3/100
18/18 [==============================] - 2s 100ms/step - loss: 0.0188 - root_mean_squared_error: 0.1369 - val_loss: 0.0018 - val_root_mean_squared_error: 0.0423
Epoch 4/100
18/18 [==============================] - 2s 102ms/step - loss: 0.0162 - root_mean_squared_error: 0.1272 - val_loss: 6.7714e-04 - val_root_mean_squared_error: 0.0260
Epoch 5/100
18/18 [==============================] - 2s 122ms/step - loss: 0.0140 - root_mean_squared_error: 0.1182 - val_loss: 7.7257e-04 - val_root_mean_squared_error: 0.0278
Epoch 6/100
18/18 [==============================] - 3s 159ms/step - loss: 0.0105 - root_mean_squared_error: 0.1022 - val_loss: 

Epoch 47/100
18/18 [==============================] - 2s 114ms/step - loss: 4.4279e-04 - root_mean_squared_error: 0.0210 - val_loss: 8.5961e-04 - val_root_mean_squared_error: 0.0293
Epoch 48/100
18/18 [==============================] - 2s 114ms/step - loss: 4.4558e-04 - root_mean_squared_error: 0.0211 - val_loss: 9.4811e-04 - val_root_mean_squared_error: 0.0308
Epoch 49/100
18/18 [==============================] - 2s 108ms/step - loss: 3.8908e-04 - root_mean_squared_error: 0.0197 - val_loss: 4.1423e-04 - val_root_mean_squared_error: 0.0204
Epoch 50/100
18/18 [==============================] - 2s 93ms/step - loss: 3.0488e-04 - root_mean_squared_error: 0.0175 - val_loss: 3.7377e-04 - val_root_mean_squared_error: 0.0193
Epoch 51/100
18/18 [==============================] - 2s 95ms/step - loss: 3.5729e-04 - root_mean_squared_error: 0.0189 - val_loss: 4.3672e-04 - val_root_mean_squared_error: 0.0209
Epoch 52/100
18/18 [==============================] - 2s 102ms/step - loss: 4.2619e-04 - roo

18/18 [==============================] - 2s 90ms/step - loss: 2.6784e-04 - root_mean_squared_error: 0.0164 - val_loss: 3.1995e-04 - val_root_mean_squared_error: 0.0179
Epoch 93/100
18/18 [==============================] - 2s 91ms/step - loss: 3.4808e-04 - root_mean_squared_error: 0.0187 - val_loss: 6.7997e-04 - val_root_mean_squared_error: 0.0261
Epoch 94/100
18/18 [==============================] - 2s 89ms/step - loss: 2.7415e-04 - root_mean_squared_error: 0.0166 - val_loss: 9.7461e-04 - val_root_mean_squared_error: 0.0312
Epoch 95/100
18/18 [==============================] - 2s 88ms/step - loss: 3.4968e-04 - root_mean_squared_error: 0.0187 - val_loss: 2.4890e-04 - val_root_mean_squared_error: 0.0158
Epoch 96/100
18/18 [==============================] - 2s 91ms/step - loss: 2.9369e-04 - root_mean_squared_error: 0.0171 - val_loss: 3.2631e-04 - val_root_mean_squared_error: 0.0181
Epoch 97/100
18/18 [==============================] - 2s 87ms/step - loss: 3.4600e-04 - root_mean_squared_er

18/18 [==============================] - 1s 78ms/step - loss: 2.9259e-04 - root_mean_squared_error: 0.0171 - val_loss: 2.8356e-04 - val_root_mean_squared_error: 0.0168
Epoch 38/100
18/18 [==============================] - 1s 76ms/step - loss: 3.1652e-04 - root_mean_squared_error: 0.0178 - val_loss: 3.7315e-04 - val_root_mean_squared_error: 0.0193
Epoch 39/100
18/18 [==============================] - 1s 77ms/step - loss: 3.8256e-04 - root_mean_squared_error: 0.0196 - val_loss: 7.7095e-04 - val_root_mean_squared_error: 0.0278
Epoch 40/100
18/18 [==============================] - 1s 77ms/step - loss: 5.4017e-04 - root_mean_squared_error: 0.0232 - val_loss: 3.6845e-04 - val_root_mean_squared_error: 0.0192
Epoch 41/100
18/18 [==============================] - 1s 76ms/step - loss: 4.0625e-04 - root_mean_squared_error: 0.0202 - val_loss: 4.6150e-04 - val_root_mean_squared_error: 0.0215
Epoch 42/100
18/18 [==============================] - 2s 89ms/step - loss: 3.4788e-04 - root_mean_squared_er

18/18 [==============================] - 1s 76ms/step - loss: 2.5159e-04 - root_mean_squared_error: 0.0159 - val_loss: 2.2129e-04 - val_root_mean_squared_error: 0.0149
Epoch 83/100
18/18 [==============================] - 1s 76ms/step - loss: 3.1928e-04 - root_mean_squared_error: 0.0179 - val_loss: 2.8595e-04 - val_root_mean_squared_error: 0.0169
Epoch 84/100
18/18 [==============================] - 1s 78ms/step - loss: 2.3678e-04 - root_mean_squared_error: 0.0154 - val_loss: 3.6979e-04 - val_root_mean_squared_error: 0.0192
Epoch 85/100
18/18 [==============================] - 1s 78ms/step - loss: 3.1186e-04 - root_mean_squared_error: 0.0177 - val_loss: 2.8391e-04 - val_root_mean_squared_error: 0.0168
Epoch 86/100
18/18 [==============================] - 1s 79ms/step - loss: 2.0931e-04 - root_mean_squared_error: 0.0145 - val_loss: 2.3809e-04 - val_root_mean_squared_error: 0.0154
Epoch 87/100
18/18 [==============================] - 1s 77ms/step - loss: 1.7389e-04 - root_mean_squared_er

,0,1,2,3
0,8.254344,8.666263,3.59755,5.3685
